<a href="https://colab.research.google.com/github/wtsyang/dl-reproducibility-project/blob/modelB-AllConv/cnnTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount Google Drive
from google.colab import drive # import drive from google colab

ROOT = "/content/drive"     # default location for the drive
print(ROOT)                 # print content of ROOT (Optional)

drive.mount(ROOT)           # we mount the google drive at /content/drive

/content/drive
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
from google.colab import files

uploaded = files.upload()

Saving Model.py to Model.py


In [0]:
from Model import Model

In [0]:
try:
    from torchsummary import summary
except ModuleNotFoundError:
    !pip install -q torchsummary
    from torchsummary import summary


In [0]:
try:
    import torch
except ModuleNotFoundError:
    from os import path
    from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
    platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

    accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

    !pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.1-{platform}-linux_x86_64.whl
    import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

In [0]:
try:
    import torchvision
except ModuleNotFoundError:
    !pip install -q torchvision

from torchvision import datasets, transforms

In [0]:
cuda = True
train_batch_size = 256
test_batch_size = 124
best_loss = float("inf")
best_epoch = -1
dataset_path = './cifar10'
gsync_save = True

In [8]:
if gsync_save:
    try:
        import utils
    except ModuleNotFoundError:
        !wget https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
        import utils

    gsync = utils.GDriveSync()

--2020-03-16 11:50:16--  https://raw.githubusercontent.com/StefOe/colab-pytorch-utils/HEAD/utils.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4519 (4.4K) [text/plain]
Saving to: ‘utils.py’

utils.py            100%[===================>]   4.41K  --.-KB/s    in 0s      

2020-03-16 11:50:16 (84.2 MB/s) - ‘utils.py’ saved [4519/4519]



In [9]:
cuda = cuda and torch.cuda.is_available()
trainset = datasets.CIFAR10(root=dataset_path, train=True, download=True)
train_mean = trainset.data.mean(axis=(0,1,2))/255  # [0.49139968  0.48215841  0.44653091]
train_std = trainset.data.std(axis=(0,1,2))/255  # [0.24703223  0.24348513  0.26158784]
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std),
])
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
train_loader = torch.utils.data.DataLoader(datasets.CIFAR10(
    root=dataset_path, train=True, download=True,
    transform=transform_train),
    batch_size=train_batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(root=dataset_path, train=False, download=True,
    transform=transform_test),
    batch_size=test_batch_size, shuffle=False, **kwargs)

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
Files already downloaded and verified


In [0]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data))

In [0]:
def test(epoch, best_loss, best_epoch, lr):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)

        output = model(data)
        # sum up batch loss
        test_loss += criterion(output, target).data
        # get the index of the max log-probability
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(
        '\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset), 100. * correct /
            len(test_loader.dataset)))
    
    if test_loss < best_loss:
        best_epoch = epoch
        best_loss = test_loss
        torch.save(model, "best_"+str(lr)+".pt")
        try:
          if gsync_save:
              gsync.update_file_to_folder("best_"+str(lr)+".pt")
        except:
          print('Failed to gsync_save.')

        try:
          model_save_name = 'best_'+str(lr)
          path = F"/content/drive/My Drive/dl-reproducibility-project/model/{model_save_name}" 
          torch.save(model.state_dict(), path+'.epoch-{}.pt'.format(epoch))
        except:
          print('Failed to save best model to personal google drive')

    return best_loss, best_epoch, correct

In [12]:
try:
  os.makedirs('/content/drive/My Drive/dl-reproducibility-project/model/')
except:
  print('')

In [13]:
for lr in [0.1,0.05,0.01]:
  print("Training with ", lr)

  
  try: 
    del model
  except:
    print('')
  
  # Maybe it's redundant 
  try:
    del criterion
    del optimizer
    del scheduler
  except:
    print('')
  
  try:
    torch.cuda.empty_cache()
  except:
    print('')
  
  #
  # Chnage Here
  #
  # Choose Model A by basemodel=[True, False, False] and choose B by basemodel=[False, True, False]
  # Choose the modification by modifiedModel=[True, False, False, False]. The first one is the model without any modification.
  # The scecond one is the strided. The third: ConvPool. The last: All CNN.
  model = Model(basemodel=[False, True, False], modifiedModel=[False, False, False, True])
  if cuda:
      model.cuda()

  criterion = nn.CrossEntropyLoss()
  optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
  scheduler = optim.lr_scheduler.MultiStepLR(
      optimizer, milestones=[200, 250, 300], gamma=0.1)
  print(model)
  summary(model.model, input_size=(3, 32, 32))
  for epoch in range(350):
      scheduler.step()
      train(epoch)
      best_loss, best_epoch, acc = test(epoch, best_loss, best_epoch, lr)
      if (epoch>100)&(acc.item()>0)&((acc.item()/len(test_loader.dataset))<=0.1):
        print("Stop at acc of test set:",acc.item()/len(test_loader.dataset))
        break

  # if in Google Colab, download your model with this
  #from google.colab import files
  #files.download("best_"+str(lr)+"pt")
  

Training with  0.1


Model(
  (model): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Conv2d(3, 96, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (2): ReLU()
    (3): Conv2d(96, 96, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Conv2d(96, 192, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): Conv2d(192, 192, kernel_size=(5, 5), stride=(2, 2), padding=(1, 1))
    (9): ReLU()
    (10): Dropout(p=0.5, inplace=False)
    (11): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU()
    (13): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
    (14): ReLU()
    (15): Conv2d(192, 10, kernel_size=(1, 1), stride=(1, 1))
    (16): ReLU()
    (17): AdaptiveAvgPool2d(output_size=1)
    (18): Flatten()
  )
  (conv_3_192_192_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv_1_192_192): Conv2d(192, 192, kernel_size=(1, 1),

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.301234
Train Epoch: 0 [25600/50000 (51%)]	Loss: 2.181620

Test set: Average loss: 0.0172, Accuracy: 1856/10000 (19%)



Uploading file best_0.1.pt: 100%|██████████| 100/100 [00:02<00:00, 40.36it/s]


Train Epoch: 1 [0/50000 (0%)]	Loss: 2.167411
Train Epoch: 1 [25600/50000 (51%)]	Loss: 1.831724

Test set: Average loss: 0.0137, Accuracy: 3631/10000 (36%)



...

Train Epoch: 51 [0/50000 (0%)]	Loss: 0.501440
Train Epoch: 51 [25600/50000 (51%)]	Loss: 0.538705

Test set: Average loss: 0.0050, Accuracy: 8100/10000 (81%)

Train Epoch: 52 [0/50000 (0%)]	Loss: 0.488719
Train Epoch: 52 [25600/50000 (51%)]	Loss: 0.588072

Test set: Average loss: 0.0059, Accuracy: 7863/10000 (79%)

Train Epoch: 53 [0/50000 (0%)]	Loss: 0.531460
Train Epoch: 53 [25600/50000 (51%)]	Loss: 0.516937

Test set: Average loss: 0.0050, Accuracy: 8036/10000 (80%)

Train Epoch: 54 [0/50000 (0%)]	Loss: 0.548178
Train Epoch: 54 [25600/50000 (51%)]	Loss: 0.502974

Test set: Average loss: 0.0050, Accuracy: 8141/10000 (81%)

Train Epoch: 55 [0/50000 (0%)]	Loss: 0.621376
Train Epoch: 55 [25600/50000 (51%)]	Loss: 0.408192

Test set: Average loss: 0.0050, Accuracy: 8072/10000 (81%)

Train Epoch: 56 [0/50000 (0%)]	Loss: 0.492897
Train Epoch: 56 [25600/50000 (51%)]	Loss: 0.564973

Test set: Average loss: 0.0050, Accuracy: 8110/10000 (81%)

Train Epoch: 57 [0/50000 (0%)]	Loss: 0.449393
Train 

Uploading file best_0.05.pt: 100%|██████████| 100/100 [00:01<00:00, 57.65it/s]


Train Epoch: 59 [0/50000 (0%)]	Loss: 0.548798
Train Epoch: 59 [25600/50000 (51%)]	Loss: 0.544433

Test set: Average loss: 0.0051, Accuracy: 8080/10000 (81%)

Train Epoch: 60 [0/50000 (0%)]	Loss: 0.559752
Train Epoch: 60 [25600/50000 (51%)]	Loss: 0.547540

Test set: Average loss: 0.0050, Accuracy: 8093/10000 (81%)

Train Epoch: 61 [0/50000 (0%)]	Loss: 0.392654
Train Epoch: 61 [25600/50000 (51%)]	Loss: 0.416193

Test set: Average loss: 0.0052, Accuracy: 8118/10000 (81%)

Train Epoch: 62 [0/50000 (0%)]	Loss: 0.402225
Train Epoch: 62 [25600/50000 (51%)]	Loss: 0.547513

Test set: Average loss: 0.0053, Accuracy: 8098/10000 (81%)

Train Epoch: 63 [0/50000 (0%)]	Loss: 0.429465
Train Epoch: 63 [25600/50000 (51%)]	Loss: 0.444442

Test set: Average loss: 0.0049, Accuracy: 8145/10000 (81%)

Train Epoch: 64 [0/50000 (0%)]	Loss: 0.520721
Train Epoch: 64 [25600/50000 (51%)]	Loss: 0.535707

Test set: Average loss: 0.0058, Accuracy: 7960/10000 (80%)

Train Epoch: 65 [0/50000 (0%)]	Loss: 0.512098
Train 

Uploading file best_0.05.pt: 100%|██████████| 100/100 [00:02<00:00, 48.25it/s]


Train Epoch: 66 [0/50000 (0%)]	Loss: 0.517807
Train Epoch: 66 [25600/50000 (51%)]	Loss: 0.458360

Test set: Average loss: 0.0058, Accuracy: 8032/10000 (80%)

Train Epoch: 67 [0/50000 (0%)]	Loss: 0.479299
Train Epoch: 67 [25600/50000 (51%)]	Loss: 0.416375

Test set: Average loss: 0.0056, Accuracy: 8005/10000 (80%)

Train Epoch: 68 [0/50000 (0%)]	Loss: 0.431846
Train Epoch: 68 [25600/50000 (51%)]	Loss: 0.505944

Test set: Average loss: 0.0052, Accuracy: 8091/10000 (81%)

Train Epoch: 69 [0/50000 (0%)]	Loss: 0.567807
Train Epoch: 69 [25600/50000 (51%)]	Loss: 0.412702

Test set: Average loss: 0.0056, Accuracy: 8036/10000 (80%)

Train Epoch: 70 [0/50000 (0%)]	Loss: 0.437229
Train Epoch: 70 [25600/50000 (51%)]	Loss: 0.453457

Test set: Average loss: 0.0053, Accuracy: 8126/10000 (81%)

Train Epoch: 71 [0/50000 (0%)]	Loss: 0.481130
Train Epoch: 71 [25600/50000 (51%)]	Loss: 0.579543

Test set: Average loss: 0.0051, Accuracy: 8169/10000 (82%)

Train Epoch: 72 [0/50000 (0%)]	Loss: 0.461887
Train 